# EDA For Found Data

Importing Needed Libraries

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk 
import re 
#import geopandas as gd

#Pandas options, reset when done 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## Merging DataFrames Together 
For this study, I will be using three datasets, one from the [World Happiness Report](https://www.worldhappiness.report/data-sharing/) depicting a survey from each country where a sample size of citizens were asked to rank their happiness on a scale from 1-10. 

In [2]:
#Read in csv from assets folder 
worldHappiness = pd.read_excel('assets/World_Happiness_2023.xls')

#Readjust some country names for easier merging 
worldHappiness['Country name'] = worldHappiness['Country name'].replace('Congo (Kinshasa)', 'Democratic Republic of the Congo')
worldHappiness['Country name'] = worldHappiness['Country name'].replace('Congo (Brazzaville)', 'Republic of the Congo')
worldHappiness['Country name'] = worldHappiness['Country name'].replace('State of Palestine', 'Palestine')
worldHappiness['Country name'] = worldHappiness['Country name'].replace('Hong Kong S.A.R. of China', 'Hong Kong')

#Rearrange row order and strip whitespaces from country names 
worldHappiness = worldHappiness.sort_values(by = 'Country name')



In [3]:
#Read in the csv file from Kaggle 
countryInformation = pd.read_csv('assets/World_Data_2023.csv')

#Readjust some country names for easier merging 
countryInformation['Country'] = countryInformation['Country'].replace('Republic of Ireland', 'Ireland')
countryInformation['Country'] = countryInformation['Country'].replace('Turkey', 'Turkiye')
countryInformation['Country'] = countryInformation['Country'].replace('Czech Republic', 'Czechia')
countryInformation['Country'] = countryInformation['Country'].replace('Palestinian National Authority', 'Palestine')

#Order by country for easier reading of raw data and strip whitespaces
countryInformation = countryInformation.sort_values(by = 'Country')
countryInformation['Country'] = countryInformation['Country'].str.strip()



In [4]:
#Read in the csv file from Kaggle 
crimeDf = pd.read_csv('assets/Crime_Rates_2023.csv')

#Rearrange the order of the countries 
crimeDf = crimeDf.rename(columns = {'country' : 'Country'})
crimeDf = crimeDf.sort_values(by = 'Country')

#Readjust some country names for easier merging and rename the column title for the country column 
crimeDf['Country'] = crimeDf['Country'].str.strip()
crimeDf['Country'] = crimeDf['Country'].replace('Turkey', 'Turkiye')
crimeDf['Country'] = crimeDf['Country'].replace('Czech Republic', 'Czechia')



In [5]:
#Putting all the of the dataframes together 


#Take in the data we want from the world happiness report dataframe 
completeDf = worldHappiness[['Country name', 'Ladder score']]
completeDf = completeDf.rename(columns = {'Country name' : 'Country', 'Ladder score' : 'Ladder Score'})
completeDf = completeDf.sort_values(by = 'Country')


#Take out the information we're interested in 
vitalInformation = countryInformation[['Country', 'Co2-Emissions', 'GDP', 'Infant mortality', 'Out of pocket health expenditure', 'Physicians per thousand', 'Unemployment rate']]

#
completeDf['Country'] = completeDf['Country'].str.strip()
completeDf = pd.merge(completeDf, crimeDf, on = 'Country')



completeDf = pd.merge(completeDf, vitalInformation, on = 'Country')


#Show results
completeDf.head()

,Country,Ladder Score,rank,crimeIndex,pop2023,Co2-Emissions,GDP,Infant mortality,Out of pocket health expenditure,Physicians per thousand,Unemployment rate
0,Afghanistan,1.8590,4,76.31,42239854.0,"8,672","$19,101,353,833",47.9,78.40%,0.28,11.12%
1,Albania,5.2773,76,42.53,2832439.0,"4,536","$15,278,077,447",7.8,56.90%,1.20,12.33%
2,Algeria,5.3291,42,52.03,45606480.0,"150,006","$169,988,236,398",20.1,28.10%,1.72,11.70%
3,Argentina,6.0237,17,63.82,45773884.0,"201,348","$449,663,446,954",8.8,17.60%,3.96,9.79%
4,Armenia,5.3417,125,22.79,2777970.0,"5,156","$13,672,802,158",11.0,81.60%,4.40,16.99%


In [6]:
print("Final DataFrame Shape: ", completeDf.shape)

Final DataFrame Shape:  (111, 11)


## Cleaning and Tidying Data

In any other project, it would be ideal to replace NaN values with something such as true or 0. However, implementing something like this for this project would be unreasonable since a 0 would very easily skew the data and as a result, causing the algorithms to produce inaccurate results. For this reason, dropping a country that doesn't have the 

In [7]:
#Drop countries that do not have enough data 
completeDf = completeDf.dropna()
completeDf = completeDf.reset_index(drop = True)
completeDf.head()

,Country,Ladder Score,rank,crimeIndex,pop2023,Co2-Emissions,GDP,Infant mortality,Out of pocket health expenditure,Physicians per thousand,Unemployment rate
0,Afghanistan,1.8590,4,76.31,42239854.0,"8,672","$19,101,353,833",47.9,78.40%,0.28,11.12%
1,Albania,5.2773,76,42.53,2832439.0,"4,536","$15,278,077,447",7.8,56.90%,1.20,12.33%
2,Algeria,5.3291,42,52.03,45606480.0,"150,006","$169,988,236,398",20.1,28.10%,1.72,11.70%
3,Argentina,6.0237,17,63.82,45773884.0,"201,348","$449,663,446,954",8.8,17.60%,3.96,9.79%
4,Armenia,5.3417,125,22.79,2777970.0,"5,156","$13,672,802,158",11.0,81.60%,4.40,16.99%


In [8]:
#Rename the columns for more clarity

completeDf = completeDf.rename(columns = {
    'Ladder Score' : 'Happiness Score (1-10)', 
    'rank' : 'Crime Ranking',
    'crimeIndex' : 'Crime Index',
    'pop2023' : 'Total Population (2023)',
    'Co2-Emissions' : 'CO2-Emissions', 
    'GDP' : 'GDP ($)', 
    'Infant mortality' : 'Infant Mortality',
    'Out of pocket health expenditure' : 'Out of Pocket Health Expenditure (%)', 
    'Physicians per thousand' : 'Physicians Per Thousand Citizens', 
    'Unemployment rate' : 'Unemployment Rate'
    })

completeDf.head()

,Country,Happiness Score (1-10),Crime Ranking,Crime Index,Total Population (2023),CO2-Emissions,GDP ($),Infant Mortality,Out of Pocket Health Expenditure (%),Physicians Per Thousand Citizens,Unemployment Rate
0,Afghanistan,1.8590,4,76.31,42239854.0,"8,672","$19,101,353,833",47.9,78.40%,0.28,11.12%
1,Albania,5.2773,76,42.53,2832439.0,"4,536","$15,278,077,447",7.8,56.90%,1.20,12.33%
2,Algeria,5.3291,42,52.03,45606480.0,"150,006","$169,988,236,398",20.1,28.10%,1.72,11.70%
3,Argentina,6.0237,17,63.82,45773884.0,"201,348","$449,663,446,954",8.8,17.60%,3.96,9.79%
4,Armenia,5.3417,125,22.79,2777970.0,"5,156","$13,672,802,158",11.0,81.60%,4.40,16.99%


In [9]:
#Check the datatypes of all columns 
completeDf.dtypes

Country                                  object
Happiness Score (1-10)                  float64
Crime Ranking                             int64
Crime Index                             float64
Total Population (2023)                 float64
CO2-Emissions                            object
GDP ($)                                  object
Infant Mortality                        float64
Out of Pocket Health Expenditure (%)     object
Physicians Per Thousand Citizens        float64
Unemployment Rate                        object
dtype: object

Ideally, it would be better if all of these values were either floats or ints. See the altering below

In [10]:
#Remove all commas and dollar signs from GDP column and convert into float 


def string_to_float(dataframe, column_name): 
    
    dataframe[column_name] = ( 
        dataframe[column_name].apply(
            lambda cell: str(cell).replace(',', '').replace('$', '').replace('%', ''))
        )

    dataframe[column_name] =  dataframe[column_name].astype(float)
    return

def string_to_int(dataframe, column_name): 
    
    dataframe[column_name] = ( 
        dataframe[column_name].apply(
            lambda cell: str(cell).replace(',', '').replace('$', '').replace('%', ''))
        )

    dataframe[column_name] =  dataframe[column_name].astype(np.int64)
    return




string_to_float(completeDf, 'GDP ($)')
string_to_float(completeDf, 'Out of Pocket Health Expenditure (%)')
string_to_float(completeDf, 'Unemployment Rate')

string_to_int(completeDf, 'CO2-Emissions')



completeDf.head()


,Country,Happiness Score (1-10),Crime Ranking,Crime Index,Total Population (2023),CO2-Emissions,GDP ($),Infant Mortality,Out of Pocket Health Expenditure (%),Physicians Per Thousand Citizens,Unemployment Rate
0,Afghanistan,1.8590,4,76.31,42239854.0,8672,1.910135e+10,47.9,78.4,0.28,11.12
1,Albania,5.2773,76,42.53,2832439.0,4536,1.527808e+10,7.8,56.9,1.20,12.33
2,Algeria,5.3291,42,52.03,45606480.0,150006,1.699882e+11,20.1,28.1,1.72,11.70
3,Argentina,6.0237,17,63.82,45773884.0,201348,4.496634e+11,8.8,17.6,3.96,9.79
4,Armenia,5.3417,125,22.79,2777970.0,5156,1.367280e+10,11.0,81.6,4.40,16.99


In [ ]:
#Much better! 
completeDf.dtypes

Country                                  object
Happiness Score (1-10)                  float64
Crime Ranking                             int64
Crime Index                             float64
Total Population (2023)                 float64
CO2-Emissions                             int64
GDP ($)                                 float64
Infant Mortality                        float64
Out of Pocket Health Expenditure (%)    float64
Physicians Per Thousand Citizens        float64
Unemployment Rate                       float64
dtype: object

## Exploratory Data Analysis (EDA)

After cleaning up the data and removing any discrepancies, we can now begin to look at common patterns shared in the data. 